## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler 
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from pandas.tools.plotting import scatter_matrix
from sklearn.decomposition import PCA
from imblearn.combine import SMOTEENN
import seaborn as sns

## 1. Load Dataset

In [2]:
data = pd.read_csv('dataset.csv',nrows = 50000) ## take the first 50000 rows

In [3]:
data.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [4]:
data['type'].value_counts()

PAYMENT     21912
CASH_OUT    13756
CASH_IN      8992
TRANSFER     4700
DEBIT         640
Name: type, dtype: int64

### 1.2 Dataset Information

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 11 columns):
step              50000 non-null int64
type              50000 non-null object
amount            50000 non-null float64
nameOrig          50000 non-null object
oldbalanceOrg     50000 non-null float64
newbalanceOrig    50000 non-null float64
nameDest          50000 non-null object
oldbalanceDest    50000 non-null float64
newbalanceDest    50000 non-null float64
isFraud           50000 non-null int64
isFlaggedFraud    50000 non-null int64
dtypes: float64(5), int64(3), object(3)
memory usage: 4.2+ MB


### 1.2 Dataset Description

In [6]:
data.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,50000.000000,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,5.000000e+04,50000.000000,50000.0
mean,7.453800,1.562645e+05,7.333083e+05,7.472741e+05,8.444671e+05,1.164400e+06,0.002000,0.0
std,2.056636,3.243949e+05,2.202405e+06,2.239971e+06,2.433388e+06,2.915203e+06,0.044677,0.0
min,1.000000,6.300000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
25%,7.000000,7.734572e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.0
50%,8.000000,3.341482e+04,1.703072e+04,0.000000e+00,2.126000e+03,0.000000e+00,0.000000,0.0
75%,9.000000,1.878390e+05,1.331716e+05,1.438734e+05,4.632050e+05,8.649264e+05,0.000000,0.0
max,9.000000,1.000000e+07,2.854724e+07,2.861740e+07,3.014320e+07,3.197699e+07,1.000000,0.0


## 2. Pre-Processing

### 2.1 Feature Engineering

In [7]:
## take the first char of the nameDest (M,C).
data['nameDest']=data['nameDest'].apply(lambda x: x[0])

In [8]:
## Select the most important Features
data = data[[ 'type', 'amount', 'oldbalanceOrg', 'newbalanceOrig','nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud']]
data.head()

,type,amount,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,PAYMENT,9839.64,170136.0,160296.36,M,0.0,0.0,0
1,PAYMENT,1864.28,21249.0,19384.72,M,0.0,0.0,0
2,TRANSFER,181.00,181.0,0.00,C,0.0,0.0,1
3,CASH_OUT,181.00,181.0,0.00,C,21182.0,0.0,1
4,PAYMENT,11668.14,41554.0,29885.86,M,0.0,0.0,0


In [9]:
## group by nameDest to see which of C or M is most frequent with isFruad
data.groupby(['nameDest']).count()['isFraud']

nameDest
C    28088
M    21912
Name: isFraud, dtype: int64

In [10]:
## count values of Is fraud which indcates that int the 50000 transactions there are 100 fraud
data['isFraud'].value_counts()

0    49900
1      100
Name: isFraud, dtype: int64

### 2.2 Missing Values Check

In [11]:
## check if the data has missing values or not
data.isnull().sum()

type              0
amount            0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
dtype: int64

### 2.3 Label Encoding

In [12]:
## convert type and nameDest values from strings to numbers
data['type'] = LabelEncoder().fit_transform(data['type'])
data['nameDest'] = LabelEncoder().fit_transform(data['nameDest'])
data.head()

,type,amount,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
0,3,9839.64,170136.0,160296.36,1,0.0,0.0,0
1,3,1864.28,21249.0,19384.72,1,0.0,0.0,0
2,4,181.00,181.0,0.00,0,0.0,0.0,1
3,1,181.00,181.0,0.00,0,21182.0,0.0,1
4,3,11668.14,41554.0,29885.86,1,0.0,0.0,0


## 3. Model Buliding

### 3.1 Split Data into Training & Test

In [13]:
features = data.iloc[:,[0,1,2,3,4,5,6]].values
labels = data.iloc[:,-1].values
validation_size = 0.3
seed = 0
scoring = 'accuracy'
x_train, x_validation, y_train, y_validation = model_selection.train_test_split(features,labels, test_size=validation_size, random_state=seed)
print("Train Size is: ",len(x_train))
print("Test Size is: ",len(x_validation))

Train Size is:  35000
Test Size is:  15000


### 3.2 Over Sampling for the training data

In [14]:
## over sample for training and testing data to increase the rows of fraud with the same chahrcterisitcs to make the data balanced
sme=SMOTEENN(random_state=42, ratio =1)
b_x_train, b_y_train = sme.fit_sample(x_train, y_train)
b_x_test, b_y_test = sme.fit_sample(x_validation, y_validation)
print(b_y_train)

[0 0 0 ..., 1 1 1]


### 3.3 Cross Validation

In [15]:
## Check Algorithms
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('RF',RandomForestClassifier()))
models.append(('NB', GaussianNB()))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10,random_state=seed)
    cv_results = model_selection.cross_val_score(model, b_x_train, b_y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

LR: 0.962173 (0.019936)
LDA: 0.877077 (0.111327)
KNN: 0.996129 (0.003222)
CART: 0.998079 (0.001035)
RF: 0.999214 (0.000664)
NB: 0.608615 (0.377735)


### 3.4 Model Building

In [16]:
## build our model with RF becuase it has the high accuracy in cross validation process
RF = RandomForestClassifier(n_estimators=5)
RF.fit(b_x_train, b_y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

## 4. Model Evaluation

In [17]:
y_pred = RF.predict(b_x_test)
print('Accuracy is : ',accuracy_score(b_y_test, y_pred))
print(confusion_matrix(b_y_test, y_pred))
print(classification_report(b_y_test, y_pred))

Accuracy is :  0.891625950054
[[14540    61]
 [ 3133 11738]]
              precision    recall  f1-score   support

           0       0.82      1.00      0.90     14601
           1       0.99      0.79      0.88     14871

   micro avg       0.89      0.89      0.89     29472
   macro avg       0.91      0.89      0.89     29472
weighted avg       0.91      0.89      0.89     29472



## 5. Model Serialization

In [18]:
## serialize our model into external file for production using
pickle.dump(RF, open('finalModel', 'wb'))